In [1]:
learning_rate = 0.01
num_steps = 20
batch_size = 20
total_numbers = 291
display_step = 1000
examples_to_show = 10
import tensorflow as tf
import numpy as np
# Network Parameters
num_hidden_1 = 256 # 1st layer num features
num_hidden_2 = 128
num_hidden_3 = 64
num_hidden_4 = 32# 2nd layer num features (the latent dim)
num_input = 41
num_classes = 3

# tf Graph input (only pictures)
X = tf.placeholder("float", [None, num_input])
Y = tf.placeholder("float", [None, num_classes])


weights = {
    'encoder_h1': tf.Variable(tf.random_normal([num_input, num_hidden_1])),
    'encoder_h2': tf.Variable(tf.random_normal([num_hidden_1, num_hidden_2])),
    'encoder_h3': tf.Variable(tf.random_normal([num_hidden_2, num_hidden_3])),
    'encoder_h4': tf.Variable(tf.random_normal([num_hidden_3, num_hidden_4])),
    'decoder_h1': tf.Variable(tf.random_normal([num_hidden_4, num_hidden_3])),
    'decoder_h2': tf.Variable(tf.random_normal([num_hidden_3, num_hidden_2])),
    'decoder_h3': tf.Variable(tf.random_normal([num_hidden_2, num_hidden_1])),
    'decoder_h4': tf.Variable(tf.random_normal([num_hidden_1, num_input])),
    'classifier_h': tf.Variable(tf.random_normal([num_hidden_4, num_classes])),
}
biases = {
    'encoder_b1': tf.Variable(tf.random_normal([num_hidden_1])),
    'encoder_b2': tf.Variable(tf.random_normal([num_hidden_2])),
    'encoder_b3': tf.Variable(tf.random_normal([num_hidden_3])),
    'encoder_b4': tf.Variable(tf.random_normal([num_hidden_4])),
    'decoder_b1': tf.Variable(tf.random_normal([num_hidden_3])),
    'decoder_b2': tf.Variable(tf.random_normal([num_hidden_2])),
    'decoder_b3': tf.Variable(tf.random_normal([num_hidden_1])),
    'decoder_b4': tf.Variable(tf.random_normal([num_input])),
    'classifier_b': tf.Variable(tf.random_normal([num_classes])),
}

In [2]:
# Building the encoder
def encoder(x):
    # Encoder Hidden layer with sigmoid activation #1
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['encoder_h1']),
                                   biases['encoder_b1']))
    # Encoder Hidden layer with sigmoid activation #2
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['encoder_h2']),
                                   biases['encoder_b2']))
    layer_3 = tf.nn.sigmoid(tf.add(tf.matmul(layer_2, weights['encoder_h3']),
                                   biases['encoder_b3']))
    layer_4 = tf.nn.sigmoid(tf.add(tf.matmul(layer_3, weights['encoder_h4']),
                                   biases['encoder_b4']))
    
    return layer_4


# Building the decoder
def decoder(x):
    # Decoder Hidden layer with sigmoid activation #1
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['decoder_h1']),
                                   biases['decoder_b1']))
    # Decoder Hidden layer with sigmoid activation #2
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['decoder_h2']),
                                   biases['decoder_b2']))
    layer_3 = tf.nn.sigmoid(tf.add(tf.matmul(layer_2, weights['decoder_h3']),
                                   biases['decoder_b3']))
    layer_4 = tf.nn.sigmoid(tf.add(tf.matmul(layer_3, weights['decoder_h4']),
                                   biases['decoder_b4']))
    return layer_4

# Construct model
encoder_op = encoder(X)
decoder_op = decoder(encoder_op)

# Prediction
y_pred = decoder_op

label_pred = tf.nn.softmax(tf.add(tf.matmul(encoder_op, weights['classifier_h']), biases['classifier_b']))
y_clipped = tf.clip_by_value(label_pred, 1e-10, 0.9999999)


# Targets (Labels) are the input data.
y_true = X
label_true = Y

# Define loss and optimizer, minimize the squared error
loss_autoencoder = tf.reduce_mean(tf.pow(y_true - y_pred, 2))
cross_entropy_loss = -tf.reduce_mean(tf.reduce_sum(label_true * tf.log(y_clipped)
                         + (1 - label_true) * tf.log(1 - y_clipped), axis=1))
loss_total = loss_autoencoder+cross_entropy_loss

optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(loss_total)

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

In [3]:
from numpy import genfromtxt
my_data = genfromtxt('./DataForBiplab/2017-2018Rabi_NDVI1.csv', delimiter=',')
my_data1 = genfromtxt('./DataForBiplab/2017-2018Rabi_VV1.csv', delimiter=',')
my_data0 = genfromtxt('./DataForBiplab/2017-2018Rabi_VH1.csv', delimiter=',')

In [4]:
data = my_data[1:,2:]
data1 = my_data1[1:,2:]
data0 = my_data0[1:,2:]
print(data.shape)
print(data1.shape)
print(data0.shape)

(301, 27)
(301, 14)
(301, 14)


In [5]:
print(data.max())
print(data1.max())
print(data0.max())

0.935826175
0.280879855
0.076032609


In [6]:

import numpy as np
data = np.array(data)
data1 = np.array(data1)
data0 = np.array(data0)
print(data.shape)
print(data1.shape)
print(data0.shape)

(301, 27)
(301, 14)
(301, 14)


In [7]:
import pandas
colnames = ['Latitude', 'Longitude', 'CropType','SampleYear','ShowingPeriod'
           , 'HarvestingPeriod', 'Others']
data3 = pandas.read_csv('./DataForBiplab/FieldDataRabi2017-2018.csv', names=colnames)

In [8]:
ctype = data3.CropType.tolist()
print(ctype)

['CropType', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat

In [9]:
p = np.array(ctype)
print(p.shape)

(302,)


In [10]:
lbl1 = ctype
label = []
for i in range(1,302):
    if(lbl1[i] == "Wheat"):
        label += [0]
    if(lbl1[i] == "Sugarcane"):
        label += [1]
    if(lbl1[i] == "Mustard"):
        label += [2]
    
print(label)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]


In [11]:
label = np.array(label)
print(label.shape)

(301,)


In [12]:
print(data.shape)
print(data1.shape)
print(data0.shape)
data11=np.log10(data1)
data00=np.log10(data0)
data4=data11/data00
data2 = np.concatenate((data,data4), axis=1)
print(data2.shape)

(301, 27)
(301, 14)
(301, 14)
(301, 41)


In [13]:
x1=[]
x2=[]
x3=[]
l1=[]
l2=[]
l3=[]
for i in range(0, label.shape[0]):
    if(label[i]==0):
        x1.append(data2[i])
        l1.append([1,0,0])
    if(label[i]==1):
        x2.append(data2[i])
        l2.append([0,1,0])
    if(label[i]==2):
        x3.append(data2[i])
        l3.append([0,0,1])
x1=np.array(x1)
x2=np.array(x2)
x3=np.array(x3)
l1=np.array(l1)
l2=np.array(l2)
l3=np.array(l3)

print(l2.shape)

(142, 3)


In [14]:
from sklearn.utils import shuffle
data2, label = shuffle(data2, label, random_state = 0)

In [15]:
# 80% train 20% test (NDVIclean)
X_train = data2[0:240,:]
X_test = data2[241:,:]
y_train = label[0:240]
y_test = label[241:]

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(240, 41)
(60, 41)
(240,)
(60,)


In [16]:
from keras.utils import np_utils
y_test = np_utils.to_categorical(y_test)
y_train = np_utils.to_categorical(y_train)
print(y_train.shape)
print(y_test.shape)

(240, 3)
(60, 3)


/home/aniruddha/anaconda3/envs/tensorflow/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [17]:
# Start Training
# Start a new TF session
sess = tf.Session()

# Run the initializer
sess.run(init)
batch_size = 20
num_batch = 12

# Training
for i in range(0,1000):
    k = 0 
    # Prepare Data
    # Get the next batch of MNIST data (only images are needed, not labels)
    avg_cost = 0
    for j in (0,num_batch):
        batch_x = X_train[k:k+batch_size,:]
        batch_y = y_train[k:k+batch_size,:]
        k += 29

    # Run optimization op (backprop) and cost op (to get loss value)
        _, l = sess.run([optimizer, loss_total], feed_dict={X: batch_x, Y: batch_y})
        avg_cost += l / num_batch
    print("Epoch:", (i + 1), "cost =", "{:.8f}".format(avg_cost))

Epoch: 1 cost = 1.35368097
Epoch: 2 cost = 0.76137441
Epoch: 3 cost = 0.45048833
Epoch: 4 cost = 0.35832013
Epoch: 5 cost = 0.31334326
Epoch: 6 cost = 0.28905222
Epoch: 7 cost = 0.27045841
Epoch: 8 cost = 0.25389630
Epoch: 9 cost = 0.23860633
Epoch: 10 cost = 0.22449112
Epoch: 11 cost = 0.21196528
Epoch: 12 cost = 0.20145715
Epoch: 13 cost = 0.19273096
Epoch: 14 cost = 0.18511851
Epoch: 15 cost = 0.17820233
Epoch: 16 cost = 0.17189812
Epoch: 17 cost = 0.16629424
Epoch: 18 cost = 0.16166070
Epoch: 19 cost = 0.15873259
Epoch: 20 cost = 0.15970822
Epoch: 21 cost = 0.17096196
Epoch: 22 cost = 0.20104423
Epoch: 23 cost = 0.21284030
Epoch: 24 cost = 0.15552160
Epoch: 25 cost = 0.12425978
Epoch: 26 cost = 0.18994309
Epoch: 27 cost = 0.14987973
Epoch: 28 cost = 0.11917979
Epoch: 29 cost = 0.13769784
Epoch: 30 cost = 0.13139655
Epoch: 31 cost = 0.13218695
Epoch: 32 cost = 0.13156135
Epoch: 33 cost = 0.11519344
Epoch: 34 cost = 0.11747898
Epoch: 35 cost = 0.13147854
Epoch: 36 cost = 0.14052789
E

Epoch: 297 cost = 0.01406173
Epoch: 298 cost = 0.01359843
Epoch: 299 cost = 0.01469810
Epoch: 300 cost = 0.01487505
Epoch: 301 cost = 0.01364513
Epoch: 302 cost = 0.01298852
Epoch: 303 cost = 0.01314815
Epoch: 304 cost = 0.01411364
Epoch: 305 cost = 0.01401328
Epoch: 306 cost = 0.01364777
Epoch: 307 cost = 0.01383798
Epoch: 308 cost = 0.01351821
Epoch: 309 cost = 0.01328188
Epoch: 310 cost = 0.01354313
Epoch: 311 cost = 0.01392733
Epoch: 312 cost = 0.01294241
Epoch: 313 cost = 0.01416621
Epoch: 314 cost = 0.01513281
Epoch: 315 cost = 0.34744510
Epoch: 316 cost = 0.58138303
Epoch: 317 cost = 0.14475054
Epoch: 318 cost = 0.03182183
Epoch: 319 cost = 0.02049812
Epoch: 320 cost = 0.01727761
Epoch: 321 cost = 0.01466906
Epoch: 322 cost = 0.01431352
Epoch: 323 cost = 0.01454436
Epoch: 324 cost = 0.01392946
Epoch: 325 cost = 0.01293631
Epoch: 326 cost = 0.01366791
Epoch: 327 cost = 0.01396268
Epoch: 328 cost = 0.01306927
Epoch: 329 cost = 0.01335568
Epoch: 330 cost = 0.01334733
Epoch: 331 cos

Epoch: 593 cost = 0.51056461
Epoch: 594 cost = 0.05575725
Epoch: 595 cost = 0.02087957
Epoch: 596 cost = 0.02238895
Epoch: 597 cost = 0.01860048
Epoch: 598 cost = 0.01327299
Epoch: 599 cost = 0.01262092
Epoch: 600 cost = 0.01263677
Epoch: 601 cost = 0.01237567
Epoch: 602 cost = 0.01244925
Epoch: 603 cost = 0.01317169
Epoch: 604 cost = 0.01342978
Epoch: 605 cost = 0.01223206
Epoch: 606 cost = 0.01252180
Epoch: 607 cost = 0.01266671
Epoch: 608 cost = 0.01242174
Epoch: 609 cost = 0.01304895
Epoch: 610 cost = 0.01264089
Epoch: 611 cost = 0.01264075
Epoch: 612 cost = 0.01259036
Epoch: 613 cost = 0.01262670
Epoch: 614 cost = 0.01263113
Epoch: 615 cost = 0.01303269
Epoch: 616 cost = 0.01303799
Epoch: 617 cost = 0.01273061
Epoch: 618 cost = 0.01288515
Epoch: 619 cost = 0.01255062
Epoch: 620 cost = 0.01266237
Epoch: 621 cost = 0.01243611
Epoch: 622 cost = 0.01282250
Epoch: 623 cost = 0.01274410
Epoch: 624 cost = 0.01215170
Epoch: 625 cost = 0.01304482
Epoch: 626 cost = 0.01292487
Epoch: 627 cos

Epoch: 884 cost = 0.01180034
Epoch: 885 cost = 0.01190337
Epoch: 886 cost = 0.01183262
Epoch: 887 cost = 0.01204321
Epoch: 888 cost = 0.01224309
Epoch: 889 cost = 0.01265439
Epoch: 890 cost = 0.01327237
Epoch: 891 cost = 0.01269714
Epoch: 892 cost = 0.01222637
Epoch: 893 cost = 0.01188181
Epoch: 894 cost = 0.01231439
Epoch: 895 cost = 0.01201016
Epoch: 896 cost = 0.01255956
Epoch: 897 cost = 0.01221351
Epoch: 898 cost = 0.01246554
Epoch: 899 cost = 0.01205665
Epoch: 900 cost = 0.01191189
Epoch: 901 cost = 0.01201157
Epoch: 902 cost = 0.01263378
Epoch: 903 cost = 0.01245559
Epoch: 904 cost = 0.01215609
Epoch: 905 cost = 0.01207890
Epoch: 906 cost = 0.01185934
Epoch: 907 cost = 0.01254063
Epoch: 908 cost = 0.01264355
Epoch: 909 cost = 0.01235580
Epoch: 910 cost = 0.01251393
Epoch: 911 cost = 0.02444353
Epoch: 912 cost = 0.25733578
Epoch: 913 cost = 0.01338433
Epoch: 914 cost = 0.01253733
Epoch: 915 cost = 0.01252870
Epoch: 916 cost = 0.01250891
Epoch: 917 cost = 0.01220351
Epoch: 918 cos

In [18]:
# define an accuracy assessment operation
correct_prediction = tf.equal(tf.argmax(Y, 1), tf.argmax(label_pred, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [19]:
print(y_test.shape)

(60, 3)


In [20]:
# on 1000 epoch
print(sess.run(accuracy, feed_dict={X: X_test, Y: y_test}))

0.8166667


In [21]:
res1=np.array(sess.run(tf.argmax(label_pred,1), feed_dict={X: x1, Y: l1}))
res2=np.array(sess.run(tf.argmax(label_pred,1), feed_dict={X: x2, Y: l2}))
res3=np.array(sess.run(tf.argmax(label_pred,1), feed_dict={X: x3, Y: l3}))
print(res1)
print(res2)
print(res3)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0
 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 1
 0 0 0 0 0 0 0 0 0]
[1 1 1 1 1 1 1 1 2 2 1 2 2 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 0 1 1 1 1 1 1 1 2 1 1 1 2 2 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 2 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 2 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[1 2 2 2 1 1 1 1 2 2 1 1 1 1 1 2 1 1 1 1 2 2 1 2 2 1 1 1 2 1 1 1 1 1 2 2 1
 1 2]


In [22]:
# confusion matrix
confusion=[]
x1=np.sum(np.equal(res1, 0))
x2=np.sum(np.equal(res1, 1))
x3=np.sum(np.equal(res1, 2))
confusion.append([x1,x2,x3])
x1=np.sum(np.equal(res2, 0))
x2=np.sum(np.equal(res2, 1))
x3=np.sum(np.equal(res2, 2))
confusion.append([x1,x2,x3])
x1=np.sum(np.equal(res3, 0))
x2=np.sum(np.equal(res3, 1))
x3=np.sum(np.equal(res3, 2))
confusion.append([x1,x2,x3])

confusion=np.array(confusion)
print(confusion)

[[110  10   0]
 [  4 127  11]
 [  0  25  14]]


In [23]:
# 70% train 30% test (NDVIclean)
# Train - 204
# Test - 87
X_train = data2[0:210,:]
X_test = data2[210:,:]
y_train = label[0:210]
y_test = label[210:]

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(210, 41)
(91, 41)
(210,)
(91,)


In [24]:
from keras.utils import np_utils
y_test = np_utils.to_categorical(y_test)
y_train = np_utils.to_categorical(y_train)
print(y_train.shape)
print(y_test.shape)

(210, 3)
(91, 3)


In [25]:
# Start Training
# Start a new TF session
sess = tf.Session()

# Run the initializer
sess.run(init)
batch_size = 20
num_batch = 10

# Training
for i in range(0,1000):
    k = 0 
    # Prepare Data
    # Get the next batch of MNIST data (only images are needed, not labels)
    avg_cost = 0
    for j in (0,num_batch):
        batch_x = X_train[k:k+batch_size,:]
        batch_y = y_train[k:k+batch_size,:]
        k += 29

    # Run optimization op (backprop) and cost op (to get loss value)
        _, l = sess.run([optimizer, loss_total], feed_dict={X: batch_x, Y: batch_y})
        avg_cost += l / num_batch
    print("Epoch:", (i + 1), "cost =", "{:.8f}".format(avg_cost))

Epoch: 1 cost = 1.19514508
Epoch: 2 cost = 0.90167203
Epoch: 3 cost = 0.75215361
Epoch: 4 cost = 0.64288328
Epoch: 5 cost = 0.53741903
Epoch: 6 cost = 0.45461054
Epoch: 7 cost = 0.40545728
Epoch: 8 cost = 0.37291979
Epoch: 9 cost = 0.34691646
Epoch: 10 cost = 0.32432413
Epoch: 11 cost = 0.30325097
Epoch: 12 cost = 0.28262184
Epoch: 13 cost = 0.26234177
Epoch: 14 cost = 0.24353799
Epoch: 15 cost = 0.22760667
Epoch: 16 cost = 0.21495297
Epoch: 17 cost = 0.20537372
Epoch: 18 cost = 0.19844595
Epoch: 19 cost = 0.19345971
Epoch: 20 cost = 0.18962034
Epoch: 21 cost = 0.18616124
Epoch: 22 cost = 0.18222791
Epoch: 23 cost = 0.17713110
Epoch: 24 cost = 0.17103971
Epoch: 25 cost = 0.16514208
Epoch: 26 cost = 0.16090089
Epoch: 27 cost = 0.15936576
Epoch: 28 cost = 0.16028430
Epoch: 29 cost = 0.15902811
Epoch: 30 cost = 0.14813563
Epoch: 31 cost = 0.13420815
Epoch: 32 cost = 0.13366821
Epoch: 33 cost = 0.15806805
Epoch: 34 cost = 0.24791594
Epoch: 35 cost = 0.73223956
Epoch: 36 cost = 0.50263035
E

Epoch: 328 cost = 0.01022815
Epoch: 329 cost = 0.01033823
Epoch: 330 cost = 0.01082759
Epoch: 331 cost = 0.01140371
Epoch: 332 cost = 0.01105357
Epoch: 333 cost = 0.01019757
Epoch: 334 cost = 0.00971562
Epoch: 335 cost = 0.01018226
Epoch: 336 cost = 0.01022964
Epoch: 337 cost = 0.01001130
Epoch: 338 cost = 0.01000320
Epoch: 339 cost = 0.01043272
Epoch: 340 cost = 0.01011165
Epoch: 341 cost = 0.01011736
Epoch: 342 cost = 0.01056978
Epoch: 343 cost = 0.26340995
Epoch: 344 cost = 0.56387041
Epoch: 345 cost = 0.31125585
Epoch: 346 cost = 0.06469992
Epoch: 347 cost = 0.04162668
Epoch: 348 cost = 0.02318472
Epoch: 349 cost = 0.01971140
Epoch: 350 cost = 0.02893530
Epoch: 351 cost = 0.11661784
Epoch: 352 cost = 0.03806616
Epoch: 353 cost = 0.02120174
Epoch: 354 cost = 0.01938365
Epoch: 355 cost = 0.01532248
Epoch: 356 cost = 0.01547058
Epoch: 357 cost = 0.01341014
Epoch: 358 cost = 0.01268244
Epoch: 359 cost = 0.01175543
Epoch: 360 cost = 0.01197828
Epoch: 361 cost = 0.01156428
Epoch: 362 cos

Epoch: 621 cost = 0.00894835
Epoch: 622 cost = 0.00871644
Epoch: 623 cost = 0.00946165
Epoch: 624 cost = 0.00935745
Epoch: 625 cost = 0.00881434
Epoch: 626 cost = 0.00847624
Epoch: 627 cost = 0.00815742
Epoch: 628 cost = 0.00842489
Epoch: 629 cost = 0.00825063
Epoch: 630 cost = 0.00826976
Epoch: 631 cost = 0.00817676
Epoch: 632 cost = 0.00873223
Epoch: 633 cost = 0.00966198
Epoch: 634 cost = 0.08719802
Epoch: 635 cost = 0.45875223
Epoch: 636 cost = 0.12597529
Epoch: 637 cost = 0.09259195
Epoch: 638 cost = 0.01751783
Epoch: 639 cost = 0.01262503
Epoch: 640 cost = 0.01073122
Epoch: 641 cost = 0.01006128
Epoch: 642 cost = 0.00941338
Epoch: 643 cost = 0.00931700
Epoch: 644 cost = 0.00969675
Epoch: 645 cost = 0.00999895
Epoch: 646 cost = 0.00920929
Epoch: 647 cost = 0.00870979
Epoch: 648 cost = 0.00885090
Epoch: 649 cost = 0.00903809
Epoch: 650 cost = 0.00921861
Epoch: 651 cost = 0.00880882
Epoch: 652 cost = 0.00869067
Epoch: 653 cost = 0.00822814
Epoch: 654 cost = 0.00945831
Epoch: 655 cos

Epoch: 913 cost = 0.05149626
Epoch: 914 cost = 0.03571373
Epoch: 915 cost = 0.02516572
Epoch: 916 cost = 0.01855972
Epoch: 917 cost = 0.01610617
Epoch: 918 cost = 0.01381744
Epoch: 919 cost = 0.01134955
Epoch: 920 cost = 0.00954423
Epoch: 921 cost = 0.00881280
Epoch: 922 cost = 0.00892793
Epoch: 923 cost = 0.00905665
Epoch: 924 cost = 0.00902392
Epoch: 925 cost = 0.00850000
Epoch: 926 cost = 0.00837696
Epoch: 927 cost = 0.00788399
Epoch: 928 cost = 0.00866794
Epoch: 929 cost = 0.00832536
Epoch: 930 cost = 0.00856875
Epoch: 931 cost = 0.00909035
Epoch: 932 cost = 0.00865856
Epoch: 933 cost = 0.00823979
Epoch: 934 cost = 0.00786579
Epoch: 935 cost = 0.00846383
Epoch: 936 cost = 0.00837577
Epoch: 937 cost = 0.00865936
Epoch: 938 cost = 0.00836296
Epoch: 939 cost = 0.00898933
Epoch: 940 cost = 0.00893077
Epoch: 941 cost = 0.00839170
Epoch: 942 cost = 0.00770664
Epoch: 943 cost = 0.00833472
Epoch: 944 cost = 0.00849707
Epoch: 945 cost = 0.00837292
Epoch: 946 cost = 0.00807750
Epoch: 947 cos

In [26]:
# on 1000 epoch
print(sess.run(accuracy, feed_dict={X: X_test, Y: y_test}))

0.84615386
